### DANGEROUS TERRITORY:
This notebook can allocate a lot of disk space and needs some processing power (ideally available CUDA GPU)

# Reddit Climate Change - Modeling Sentiment & Emotion
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [27]:
from transformers import pipeline # Protobuf version <4 (e.g. 3.20.3) might be needed!

import pandas as pd
import numpy as np
import os
import torch
import pickle
from tqdm import tqdm
import time
from datetime import datetime
import sys
import multiprocessing.dummy as mp 
import logging

In [2]:
print(f"CUDA device found: {torch.cuda.get_device_name(torch.cuda.current_device())}")
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    batch_size = 8
    print("### \n WARNING: YOU WILL TRAIN ON DETECTED GPU \n###")
else:
    batch_size = 1
    device = -1

CUDA device found: NVIDIA GeForce GTX 1060 6GB
### 
###


## Load data

In [4]:
# Creation of final dataset

# df2 = pd.read_csv("data/comments2.csv", header=0)
# df2 = df2.dropna()
# df2.head(3)

# df = df.sort_values(by="id").reset_index()
# df2 = df2.sort_values(by="id").reset_index()

# df["body_clean_full"] = df2.body_clean
# df["id_2"] = df2.id
# print(sum(df.id != df.id_2))

# df = df.drop(["index", "id_2", "body_clean"], axis=1)
# df["body_clean_full"] = [i.replace("\n", " ") for i in df.body_clean_full]


# Load compressed data
#df = pd.read_csv("data/comments_preprocessed.gzip", compression="gzip", header=0, skiprows=0, nrows=11000)

In [3]:
# Load comments
df = pd.read_csv("data/comments_final.csv", header=0, index_col=0)#

# Sanity check
if not df[df.isna().any(axis=1)].empty:
    raise Exception("Sanity check failed! Empty rows detected!")

df.head(3)

,id,subreddit.name,subreddit.nsfw,created_utc,permalink,sentiment,score,created_date,created_day,created_month,created_year,created_time,topic_number,topic_name,topic_most_used_words,body_clean_full
0,c0i14fb,askreddit,False,1.262306e+09,https://old.reddit.com/r/AskReddit/comments/ak...,0.7998,1.0,2010-01-01,1,1,2010,00:34:07,-1,-1_climate_people_global_warming,climate - people - global - warming - just - s...,"should be ""San Diego Weatherman has an opinion..."
1,c0i195b,worldnews,False,1.262313e+09,https://old.reddit.com/r/worldnews/comments/ak...,0.4754,0.0,2010-01-01,1,1,2010,02:30:18,0,0_people_just_climate_global,people - just - climate - global - don - like ...,Both Iggy and Harper would have marched us int...
2,c0i1a0w,environment,False,1.262314e+09,https://old.reddit.com/r/environment/comments/...,0.0242,1.0,2010-01-01,1,1,2010,02:54:40,0,0_people_just_climate_global,people - just - climate - global - don - like ...,"A man who though a moderate Tory , has a mixed..."


In [5]:
df.dtypes

id                        object
subreddit.name            object
subreddit.nsfw              bool
created_utc              float64
permalink                 object
sentiment                float64
score                    float64
created_date              object
created_day                int64
created_month              int64
created_year               int64
created_time              object
topic_number               int64
topic_name                object
topic_most_used_words     object
body_clean_full           object
dtype: object

In [6]:
df.shape

(1041570, 16)

In [12]:
# Extract sample text
df.body_clean_full.iloc[21]

"Not necessarily.  He could simply be [sympathetic to the environmentalism movement.]( .com/communities/greenhouse/post/ / /james-cameron-sees-avatar-as-environmental-warning/ )  Edit: Imagine a hypothetical scenario, if you will.    An environmentalist friend of Cameron calls him up for a chat:  Environmentalist: Hey James, you know that movie you've been thinking about doing; the one about the evil corporation who's destroying the planet for greed, and how the planet fights back in the end and wins?  James: Yeah?  Environmentalist: Well, I think that if you went ahead and did that, it could have a great effect on people right now.  We're trying to save the earth, and people just aren't really concerned enough for us to REALLY be able to make a difference.  We have the Copenhagen conference coming up at the end of  , and that will mark the beginning of the big push for real, meaningful change to save the environment.  The problem is that we just aren't quite sure that the American pub

Faulty rows of old preprocessed_comments file: 

368464 - 393215:    "i38o4bg	climatechange	False	1649003948	"

393217 - 425983:    "0.0	[AR IPCC Chapter : Understanding and Attribu...	NaN"



## HF Transformers Models

Be aware: Models rather large, first time to run might take some downloading time (~500MB per model; saved in sth like "C:\Users\Felix\.cache\huggingface\hub")

All HF sentiment models: https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads&search=sentiment

All HF emotion detection models: https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads&search=emotion

### Sentiment Model

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

ENCODING: 0 -> Negative; 1 -> Neutral; 2 -> Positive

GPU: ~0.0105 sec/text

In [24]:
model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", device=device, batch_size=batch_size)
model_name = "sentiment"

### Climate stance model

https://huggingface.co/cardiffnlp/twitter-roberta-base-stance-climate

Paper: https://aclanthology.org/S16-1003.pdf

"Climate Change is a Real Concern" -> favor/against/none

GPU: ~0.014 sec/text

In [7]:
model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-stance-climate", device=device, batch_size=batch_size)
model_name = "climate_stance"

### Climate sentiment model

https://huggingface.co/climatebert/distilroberta-base-climate-sentiment 

-> neutral, opportunity, risk

In [ ]:
model = pipeline("sentiment-analysis", model="climatebert/distilroberta-base-climate-sentiment", device=device, batch_size=batch_size)
model_name = "climate_sentiment"

### Large emotion model (28 states detected)

Model: https://huggingface.co/arpanghoshal/EmoRoBERTa

Labels detected: 'remorse', 'disappointment', 'sadness', 'gratitude', 'realization', 'disapproval', 'neutral', 'approval', 'embarrassment', 'caring', 'curiosity', 'confusion', 'annoyance', 'joy', 'optimism', 'relief', 'excitement', 'admiration', 'love', 'disgust', 'grief', 'amusement', 'anger', 'surprise', 'pride', 'nervousness', 'fear', 'desire'

GPU: ~0,11 sec/text

In [6]:
model = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa', top_k=1, device=device, batch_size=batch_size) # top_k=None lists all labels
model_name = "emotion_large"

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


### Small emotion model (7 states)

Model: https://huggingface.co/j-hartmann/emotion-english-distilroberta-base

Labels detected: 'surprise', 'neutral', 'fear', 'anger', 'joy', 'disgust', 'sadness'

GPU: ~0,011 sec/text

In [4]:
model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1, device=device, batch_size=batch_size) # top_k=None lists all labels
model_name = "emotion_small"

## Test model functionality

In [8]:
sample_prediction = model("Climate change is a big scam! Why is everyone so upset?!?!")
sample_prediction

[{'label': 'favor', 'score': 0.9356362819671631}]

In [ ]:
# Find labels included
#labels = [i["label"] for i in sample_prediction]
#print(labels)

## Apply model manually

In [26]:
# Create backup of df
#df_backup = df.copy()

In [9]:
# Reduce text to character limit if there is one
def reduce_text(text, max_length=np.inf):
    if len(text) > max_length:
        text = text[0:max_length]
    return text

In [10]:
# Reduce texts if necessary
df["body_clean_full"] = df["body_clean_full"].apply(reduce_text, args=(512, )) # e.g. for small sentiment model
print("Comment text reduced to 512 characters!")

Comment text reduced to 512 characters!


In [11]:
# Only use subset
df.sample(frac=1, random_state=42) # shuffle
df = df.iloc[:1000, :]

In [11]:
# CAUTION: THIS WILL RUN INFERENCE AND CAN THUS TAKE SOME TIME
#df["label"] = model(list(df["body_clean_full"]))
results = df.body_clean_full.iloc[:1000].map(model)

c:\ProgramData\Anaconda3\lib\site-packages\transformers\pipelines\base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [14]:
df.body_clean_full.iloc[:1000]

0      should be "San Diego Weatherman has an opinion...
1      Both Iggy and Harper would have marched us int...
2      A man who though a moderate Tory , has a mixed...
3      Changing the oil *filter* every single time yo...
4      ; We have no history - ours goes back only   y...
                             ...                        
995    take a look at the presidential voting record ...
996    Sadly, I think she's spot on. As I posted in m...
997    The Intergovernmental Panel on **Climate Chang...
998    i was looking merely at a correlation between ...
999    India has criticised the UN panel on climate c...
Name: body_clean_full, Length: 1000, dtype: object

In [25]:
# Recommended inference on GPU
# https://huggingface.co/docs/transformers/pipeline_tutorial#using-pipelines-on-a-dataset
def load_iterator():
    for i in df.body_clean_full.iloc[:1000]:
        # DEBUGGING
        # i = i.replace("\"", " ")
        # i = i.replace("'", " ")
        # print(i)
        # print("")
        yield i

results = []

for out in model(load_iterator()):
    results.append(out)

In [18]:
# Extract (top) label
results = [i[0]["label"] for i in results]

In [13]:
# Print all top labels found
np.unique(results)

array(['favor', 'none'], dtype='<U5')

In [59]:
# Get sample texts for certain label
print(df[df.top_label == "optimism"].body_clean_full.iloc[1])

; We can't really predict what kind of technology we'll have in   years.  Ah, yes - the magical scientific solution to everything that might, hopefully get invented. The catch-all excuse to justify any behaviour now. Trouble is, it won't replace the species and biodiversity that has been lost. Nor will it prevent the death of millions as drought creeps across the planet.  ; Yeah, the road thing is an understatement - we'd have to rework our docks  Jesus. You've gone from moving roads to reworking docks only


### Run prediction per year

In [26]:
# Only use subset
# df = df.sample(frac=1, random_state=42) # shuffle
# df = df.iloc[:1000, :]

In [6]:
# Reduce texts if necessary
df["body_clean_full"] = df["body_clean_full"].apply(reduce_text, args=(512, )) # e.g. for small sentiment model
print("Comment text reduced to 512 characters!")

Comment text reduced to 512 characters!


In [14]:
df.created_year.unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022], dtype=int64)

In [19]:
# Start logging
# https://docs.python.org/3/howto/logging.html
folder = f"data/{model_name}_labels/"
if not os.path.exists(folder):
    os.mkdir(folder)

logging.basicConfig(filename=f"data/{model_name}_labels/{model_name}_logs.log", level=logging.DEBUG)

In [20]:
def save_file(save_df, idx, to_csv=False):
    idx = str(idx)
    folder = f"data/{model_name}_labels/"
    path = folder + f"{model_name}_{idx}"

    if os.path.exists(path + ".pkl"): 
        path = path + "_" + str(datetime.now())[-5:]
        logging.info(str(datetime.now()) + f" Warning: path already existed, will save as: {path}")
    if not os.path.exists(folder):
        os.mkdir(folder)

    try:
        with open(path + ".pkl", "wb") as f:
            pickle.dump(save_df, f)
        logging.info(str(datetime.now()) + f" Year {idx} saved in {path}")
    except:
        # Save to csv if failed
        try:
            logging.info(str(datetime.now()) + " Pickling failed, will try to save as csv")
            save_df.to_csv(path + "_BACKUP.csv", index=False)
        except:
            logging.info(str(datetime.now()) + f" ERROR: Failed to save year {idx}!")
            logging.info(str(sys.exc_info()))
            return False

    return True

In [14]:
# Sanity check
#save_file(df, 2004)

In [87]:
# Remove quotes
# df.body_clean_full = [i.replace(" \"", "\\\"") for i in df.body_clean_full]
# df.body_clean_full = [i.replace("'", " ") for i in df.body_clean_full]

In [21]:
max_length = 512

def executer(df_sample, year):
    try:
        logging.info(str(datetime.now()) + f" Running inference for year {year}, with {df_sample.shape[0]} samples")

        # Run inference as recommended for GPU
        # https://huggingface.co/docs/transformers/pipeline_tutorial#using-pipelines-on-a-dataset
        def load_iterator():
            for i in df_sample.body_clean_full: # FIXME DEBUGGING .iloc[83814:]
                yield i[0:max_length]

        labels = []
        idx = 0
        for idx, out in enumerate(model(load_iterator())): # Run inference and collect labels
            last_idx = idx
            labels.append(out)

        #labels = model(list(df_sample["body_clean_full"])) # OLD WAY TO RUN INFERENCE

        #labels = [i[0]["label"] for i in labels]                                # FIXME for: emotion_small
        labels = [i["label"] for i in labels]                                   # FIXME for: climate_stance

        df_save = pd.DataFrame({"id": df_sample.id, model_name: labels})
        save_file(df_save, year)

    except:
        logging.info(str(datetime.now()) + f" ERROR with year {year}: \n" + str(sys.exc_info()))
        logging.info(str(datetime.now()) + f" Last idx checked: {idx}, with id: {df_sample.iloc[idx,:].id}")

In [20]:
def executer2(df_sample, year):
    try:
        logging.info(str(datetime.now()) + f" Running inference for year {year}, with {df_sample.shape[0]} samples")

        # Run inference in basic way
        labels = df.body_clean_full.map(model)
        
        labels = [i[0]["label"] for i in labels] # FIXME ADAPT PER MODEL

        df_save = pd.DataFrame({"id": df_sample.id, model_name: labels})
        save_file(df_save, year)

    except:
        logging.info(str(datetime.now()) + f" ERROR with year {year}: \n" + str(sys.exc_info()))

In [46]:
# Sanity check
#executer(2021)

In [ ]:
# FIXME FIXME FIXME
# TODO remove duplicates from already inferred data for emotion_small!

In [22]:
# Sanity check
df[df.body_clean_full.apply(len) > 512]

,id,subreddit.name,subreddit.nsfw,created_utc,permalink,sentiment,score,created_date,created_day,created_month,created_year,created_time,topic_number,topic_name,topic_most_used_words,body_clean_full


In [23]:
# RUN INFERENCE ON ALL DATA

years = df.created_year.unique()
#years = [2015]#, 2021] # FIXME DEBUGGING
years[years.sort()]

logging.info(f"Running inference for {model_name} \n")
print(datetime.now(), f"Running inference for {model_name} \n")

for year in tqdm(years):
    df_sample = df[df.created_year == year]
    
    executer(df_sample, year) # optimized way through iterator
    #executer2(df_sample, year) # old way

logging.info(f"Inference finished for {model_name}")

2023-05-13 12:03:58.409510 Running inference for climate_stance 



  0%|          | 0/13 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\lib\site-packages\transformers\pipelines\base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 13/13 [2:27:03<00:00, 678.74s/it] 


In [8]:
df.body_clean_full[df.created_year == 2015].iloc[83815]

'Climate change is the biggest threat and we have already acknowledged the danger that IS poses. Mentioning paris would have seemed redundant and it doesnt focus on the impending dangers to come. Yes paris was shitty but his point mentioning the climate change link was acknowledging that this issue with global terrorism and conflict has the potential to get much much worse unless it is made a central issue.'

In [52]:
# Remove problem posing rows
ids = ["cx2fqug", "cnmkpqk"]

df = df[~df.id.isin(ids)]

In [85]:
df[df.id == "cnmkpqk"].body_clean_full.values[0]

'That\'s because so few people in the developed world are going to call themselves a "climate change refugee."  "So, why did you move?"  Answers that may well mean you are climate change refugee, without having to formally acknowledge that fact to yourself or others:  "We moved for better weather."  "We moved to get away from all the snow."  "We moved to lower our utility bills.  You wouldn\'t believe how expensive water in [blank] has gotten!  Greedy government, gouging working people!"  "We moved because tho'

In [15]:
# years = df.created_year.unique()

# start_time = time.localtime() # Save timestamps to show starting time and runtime
# start_t = time.time()
# print(datetime.now(), "Inference started")

# # Run inference in parallel
# p = mp.Pool() # without argument uses: os.cpu_count(), i.e. max number of threads
# p.map_async(executer, years)
# p.close()
# p.join()

# end_t = time.time()
# print(datetime.now(), "Computation time (Min.):", (end_t-start_t)/60)

## Load results

In [127]:
# Load file
with open(f"data/{model_name}_labels/{model_name}_2021.pkl", "rb") as f:
    labels_df = pickle.load(f)

In [128]:
labels_df

,id,emotion_small
1069123,hpsc48q,neutral
1052862,hiizdql,neutral
1032030,haa9dbm,disgust
981644,gnridwg,joy
990338,gs9y8eh,neutral
...,...,...
996821,gvwmgp9,neutral
1027574,h8plyab,joy
1035834,hbrxh49,neutral
1008584,h2rbqc6,neutral


## Join labels to full results

In [ ]:
# TODO